In [4]:
import pandas as pd

X_train,X_test,y_train,y_test = pd.read_csv('X_train.csv'),pd.read_csv('X_test.csv' ),pd.read_csv('y_train.csv'),pd.read_csv('y_test.csv')
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, average_precision_score, log_loss

In [2]:
%pip install optuna

Note: you may need to restart the kernel to use updated packages.


In [11]:
import optuna
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

def objective(trial):
    # Define hyperparameters to be optimized
    r = trial.suggest_float('l1_ratio', 0, 1, log=False)
    c = trial.suggest_float('C', 1e-10, 1000, log=True)
    
    # Create logistic regression model with specified hyperparameters
    model = LogisticRegression(penalty='elasticnet',solver='saga', l1_ratio=r, C=c)
    
    # Evaluate model using cross-validation
    cv_score = cross_val_score(model, X_train, y_train, cv=5,scoring = "neg_log_loss")
    
    # Return negative mean accuracy as Optuna aims to minimize the objective
    return -cv_score.mean()

# Create Optuna study
study = optuna.create_study(direction='minimize')

# Optimize hyperparameters
study.optimize(objective, n_trials=100)

# Get best hyperparameters
best_params = study.best_params
print("Best hyperparameters:", best_params)

# Train logistic regression classifier with best hyperparameters
best_model = LogisticRegression(**best_params, solver='saga', random_state=42)
best_model.fit(X_train, y_train)

# Evaluate on test set
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)

[I 2024-04-24 09:33:13,949] A new study created in memory with name: no-name-d74fb9c5-92bb-49e4-985c-2955cdeb8aa5


/home/jordigb/.pyenv/versions/3.10.4/envs/PLH/lib/python3.10/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/jordigb/.pyenv/versions/3.10.4/envs/PLH/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/jordigb/.pyenv/versions/3.10.4/envs/PLH/lib/python3.10/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/jordigb/.pyenv/versions/3.10.4/envs/PLH/lib/python3.10/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Ple

Best hyperparameters: {'l1_ratio': 0.9614205218365002, 'C': 0.15623979394035864}
Test Accuracy: 0.7966398319915996


In [ ]:
import tensorflow as tf
import optuna

# Define the function to build and train the neural network
def build_model(trial):
    # Define hyperparameters to optimize
    num_layers = trial.suggest_int('num_layers', 1, 5)
    num_units = trial.suggest_int('num_units', 16, 256)

    # Define the model architecture
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=(input_shape,)))  # Assuming input_shape is defined
    for i in range(num_layers):
        model.add(tf.keras.layers.Dense(num_units, activation='relu'))
    model.add(tf.keras.layers.Dense(2, activation='softmax'))  # Assuming num_classes is defined
    
    # Compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Define the objective function to be optimized by Optuna
def objective(trial):
    # Build the model
    model = build_model(trial)
    
    # Train the model
    model.fit(X_train, y_train, epochs=1000, batch_size=4098, verbose=0)
    
    # Evaluate the model on the validation set
    score = model.evaluate(validation_data, validation_labels, verbose=0)
    
    # Return the validation accuracy as the objective value to optimize
    return score[1]

# Define your data and constants such as input_shape, num_classes, train_data, train_labels, 
# validation_data, validation_labels, num_epochs, batch_size before proceeding.

# Run the optimization process
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Get the best hyperparameters
best_params = study.best_params
best_accuracy = study.best_value

print("Best hyperparameters:", best_params)
print("Best accuracy:", best_accuracy)